# Travel Data Analysis

### Importing Required Libraries

In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Travel Data Analysis using Pyspark").getOrCreate()

### Declaring Schema and Importing Required Datasets

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
custom_schema = StructType([
    StructField("CityPair", StringType(), True),
    StructField("FromLocation", StringType(), True),
    StructField("ToLocation", StringType(), True),
    StructField("ProductType", IntegerType(), True),
    StructField("AdultsTraveling", IntegerType(), True),
    StructField("SeniorsTraveling", IntegerType(), True),
    StructField("ChildrenTraveling", IntegerType(), True),
    StructField("YouthTraveling", IntegerType(), True),
    StructField("InfantTraveling", IntegerType(), True),
    StructField("DateOfTravel", StringType(), True),
    StructField("DateOfReturn", StringType(), True),
    StructField("AirlinePrice", FloatType(), True),
    StructField("CarPrice", FloatType(), True),
    StructField("HotelPrice", FloatType(), True),
    StructField("HotelName", StringType(), True)
])

df = spark.read.format("csv") \
    .option("delimiter", "\t") \
    .schema(custom_schema) \
    .load("/FileStore/tables/TravelData/TravelDataset.txt")
df.show()

+--------+------------+----------+-----------+---------------+----------------+-----------------+--------------+---------------+--------------------+--------------------+------------+--------+----------+---------+
|CityPair|FromLocation|ToLocation|ProductType|AdultsTraveling|SeniorsTraveling|ChildrenTraveling|YouthTraveling|InfantTraveling|        DateOfTravel|        DateOfReturn|AirlinePrice|CarPrice|HotelPrice|HotelName|
+--------+------------+----------+-----------+---------------+----------------+-----------------+--------------+---------------+--------------------+--------------------+------------+--------+----------+---------+
| ZIH-ZIH|         ZIH|       ZIH|          4|              2|               0|                0|             0|              0|2014-10-23 00:00:...|2014-10-25 00:00:...|         0.0|     0.0|    2003.2|     null|
| ZIH-ZIH|         ZIH|       ZIH|          4|              2|               0|                0|             0|              0|2014-10-23 00:00

### Converting Dataframe to Temporary View

In [0]:
df.createOrReplaceGlobalTempView("Travel")
result=spark.sql("SELECT * FROM global_temp.Travel");
result.show()

+--------+------------+----------+-----------+---------------+----------------+-----------------+--------------+---------------+--------------------+--------------------+------------+--------+----------+---------+
|CityPair|FromLocation|ToLocation|ProductType|AdultsTraveling|SeniorsTraveling|ChildrenTraveling|YouthTraveling|InfantTraveling|        DateOfTravel|        DateOfReturn|AirlinePrice|CarPrice|HotelPrice|HotelName|
+--------+------------+----------+-----------+---------------+----------------+-----------------+--------------+---------------+--------------------+--------------------+------------+--------+----------+---------+
| ZIH-ZIH|         ZIH|       ZIH|          4|              2|               0|                0|             0|              0|2014-10-23 00:00:...|2014-10-25 00:00:...|         0.0|     0.0|    2003.2|     null|
| ZIH-ZIH|         ZIH|       ZIH|          4|              2|               0|                0|             0|              0|2014-10-23 00:00

### Problem 1
Top 20 destination people travel the most: Based on the given data, we can find the most popular destination that people travel frequently. There are many destinations out of which we will find only first 20, based on trips booked for particular destinations.

In [0]:
result=spark.sql("select ToLocation, count(*) as Frequency from global_temp.Travel group by ToLocation order by Frequency desc limit 20")
result.show()

+----------+---------+
|ToLocation|Frequency|
+----------+---------+
|       MIA|      396|
|       SFO|      290|
|       LAS|      202|
|       LAX|      162|
|       DFW|      102|
|       DEN|       64|
|       ORD|       57|
|       PHL|       54|
|       IAH|       50|
|       JFK|       45|
|       PHX|       44|
|       FLL|       40|
|       ATL|       36|
|       BOS|       31|
|       MCO|       31|
|       SAN|       27|
|       WAS|       25|
|       CUN|       24|
|       AUS|       22|
|       LON|       22|
+----------+---------+



### Problem 2
Top 20 locations from where people travel the most: We can find the places from where most of the trips are undertaken, based on the booked trip count.

In [0]:
result=spark.sql("select ToLocation, count(*) as Frequency from global_temp.Travel group by ToLocation order by Frequency desc limit 20")
result.show()

+----------+---------+
|ToLocation|Frequency|
+----------+---------+
|       MIA|      396|
|       SFO|      290|
|       LAS|      202|
|       LAX|      162|
|       DFW|      102|
|       DEN|       64|
|       ORD|       57|
|       PHL|       54|
|       IAH|       50|
|       JFK|       45|
|       PHX|       44|
|       FLL|       40|
|       ATL|       36|
|       BOS|       31|
|       MCO|       31|
|       SAN|       27|
|       WAS|       25|
|       CUN|       24|
|       AUS|       22|
|       LON|       22|
+----------+---------+



### Problem 3
Top 20 cities that generate high airline revenues for travel, so that the site can concentrate on offering discount on booking, to those cities to attract more bookings.

In [0]:
result=spark.sql("select ToLocation, sum(AirlinePrice) as Airline_Revenue from global_temp.Travel group by ToLocation order by Airline_Revenue desc limit 20")
result.show()

+----------+------------------+
|ToLocation|   Airline_Revenue|
+----------+------------------+
|       SFO|192705.82097625732|
|       MIA|180603.39154052734|
|       MLE|      175974.59375|
|       DFW| 136412.6213531494|
|       LAS| 96924.08037567139|
|       DEN| 89332.37953186035|
|       AUH| 48863.29931640625|
|       IAH| 41028.40982055664|
|       LON|   40217.599609375|
|       BGO|38432.599853515625|
|       FRA| 36816.04089355469|
|       LAX| 36143.41011810303|
|       JFK|28638.589920043945|
|       BOS| 26488.99981689453|
|       NYC| 22120.86016845703|
|       MEM|21182.300170898438|
|       ZRH|20985.759887695312|
|       AMS|20306.599243164062|
|       LGW|    18146.19921875|
|       ORD| 18007.17010498047|
+----------+------------------+

